In [1]:
import numpy as np 
import pandas as pd 
import os 
from decouple import config
from pycelonis import get_celonis

CELONIS_URL = config('CELONIS_URL')
API_TOKEN = config('API_TOKEN')

celonis = get_celonis(celonis_url= CELONIS_URL,api_token= API_TOKEN)
print("Successsull Login")


2022-06-15 17:12:41 - pycelonis: Login successful! Hello Moreno Maria
Successsull Login


In [2]:
df = pd.read_csv(config('OUTPUT_FILE_NAME') + '.csv')

column_config = [{'fieldLength': 20, 'columnName': 'resource', 'columnType': 'STRING'}, {'fieldLength': 20, 'columnName': 'status', 'columnType': 'STRING'}, {'columnName': 'case_id', 'columnType': 'INTEGER'}, {'columnName': 'timestamp', 'columnType': 'DATE'}, {'fieldLength': 20, 'columnName': 'activity', 'columnType': 'STRING'}]
column_config_grades = [{'columnName': 'case_id', 'columnType': 'STRING'}, {'columnName': 'final_mark', 'columnType': 'FLOAT'}]

In [3]:
print(config('CELONIS_DATA_POOL_NAME'))
dp = celonis.pools
   
if config('CELONIS_DATA_POOL_NAME') not in dp.names:
    celonis.create_pool(config('CELONIS_DATA_POOL_NAME'))
    dp = celonis.pools
    
data_pool = dp.find(config('CELONIS_DATA_POOL_NAME'))  
print(data_pool)
print("data_pool configured")


data_pool_test
<Pool, id ac0e4ab2-c0a7-434b-a9f0-11c618d191b5, name data_pool_test>
data_pool configured


In [4]:
print(config('CELONIS_DATA_MODEL_NAME'))
dm = celonis.datamodels

if config('CELONIS_DATA_MODEL_NAME') not in dm.names:
    celonis.create_datamodel(config('CELONIS_DATA_MODEL_NAME'),data_pool)
    dm = celonis.datamodels

data_model = dm.find(config('CELONIS_DATA_MODEL_NAME'))  
print(data_model)
print("data_model configured")


data_model_test
<Datamodel, id 8ccd34da-4ffa-4bf6-9f43-1b49db348838, name data_model_test>
data_model configured


In [5]:
print(config('CELONIS_WORKSPACE_NAME'))
ws = celonis.workspaces

if config('CELONIS_WORKSPACE_NAME') not in ws.names:
    celonis.create_workspace(config('CELONIS_WORKSPACE_NAME'), data_model)
    ws = celonis.workspaces

work_space = ws.find(config('CELONIS_WORKSPACE_NAME'))  
print(work_space)
print("work_space configured")


workspace_test
<Workspace, id 0dcb5d5d-6f01-42a2-9f5c-58824f8bb0b1, name workspace_test>
work_space configured


In [6]:
analyses = celonis.analyses

if config('CELONIS_ANALYSIS_NAME') not in analyses.names:
    celonis.create_analysis(config('CELONIS_ANALYSIS_NAME'),work_space)
    analyses = celonis.analyses

analysis = analyses.find(config('CELONIS_ANALYSIS_NAME'))
print(analysis)
print("analysis configured")


<Analysis, id d32d6e7f-7df0-4213-8632-c945b5d7c8b7, name analysis_test>
analysis configured


In [7]:
data_pool.create_table(table_name=config('CELONIS_TABLE_NAME'),
                       df_or_path=df,
                       if_exists="drop",
                       column_config=column_config)

if config('CELONIS_TABLE_NAME') in data_model.tables.names:
    data_model_table = data_model.tables.find(config('CELONIS_TABLE_NAME'))
    data_model_table.delete()
    data_model = dm.find(config('CELONIS_DATA_MODEL_NAME')) 

data_model.add_table_from_pool(table_name=config('CELONIS_TABLE_NAME'), 
                               alias= config('CELONIS_TABLE_NAME'),
                               reload="FORCE_COMPLETE")

data_model.create_process_configuration(
        case_table=None,
        activity_table=config('CELONIS_TABLE_NAME'),
        case_column="case_id",
        activity_column="activity",
        timestamp_column="_CELONIS_CHANGE_DATE",
        sorting_column="status")

data_model.reload(tables=config('CELONIS_TABLE_NAME'))

print("Events Logs sent and Model configured, Analysis available in celonis platform")

2022-06-15 17:13:19 - pycelonis: Dropping existing table table_test_logs.
2022-06-15 17:13:21 - pycelonis: Data push job started...
2022-06-15 17:13:25 - pycelonis: Data push job status finished: DONE
2022-06-15 17:13:27 - pycelonis: Complete Data Model reload started...
2022-06-15 17:13:41 - pycelonis: Partial Data Model reload started...
2022-06-15 17:14:11 - pycelonis: Partial Data Model reload running...
2022-06-15 17:14:41 - pycelonis: Partial Data Model reload running...
Events Logs sent and Model configured, Analysis available in celonis platform
